In [1]:
import keras
import os
import numpy as np

base_dir = 'drive/My Drive/Colab Notebooks/Nota_Assignment'

train_dir = os.path.join(base_dir, 'train_data')
valid_dir = os.path.join(base_dir, 'valid_data')

np.random.seed(3)

Using TensorFlow backend.


In [2]:
callbacks_list = [
            keras.callbacks.EarlyStopping(
                monitor='val_accuracy',
                patience=5,
            ),
            keras.callbacks.ModelCheckpoint(
                filepath='./drive/My Drive/Colab Notebooks/Nota_Assignment/my_model.h5',
                monitor = 'val_loss',
                save_best_only=True,
            )   
]

In [3]:
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [4]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
               padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax', name='predictions')(x)

    model = Model(img_input, output)
    return model

In [5]:
input_shape = (64, 64, 1)
num_classes = 5
model = mini_XCEPTION(input_shape, num_classes)

In [6]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 62, 62, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 62, 62, 8)    0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode="nearest")

test_datagen = ImageDataGenerator(
    rescale=1./255,
    )

In [8]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        valid_dir,
        target_size=(64,64),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

Found 1695 images belonging to 5 classes.
Found 421 images belonging to 5 classes.


In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=1695//32*100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=421//32,
      callbacks=callbacks_list
      )

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 28 could not be retrieved. It could be because a worker has died.
  UserWarning)


   5/5200 [..............................] - ETA: 43:18:28 - loss: 1.9138 - accuracy: 0.2937

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 35 could not be retrieved. It could be because a worker has died.
  UserWarning)


   7/5200 [..............................] - ETA: 43:18:47 - loss: 1.8716 - accuracy: 0.3080

...

  33/5200 [..............................] - ETA: 43:03:15 - loss: 1.7359 - accuracy: 0.2900

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 12 could not be retrieved. It could be because a worker has died.
  UserWarning)


  45/5200 [..............................] - ETA: 42:24:04 - loss: 1.6983 - accuracy: 0.2981

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 5 could not be retrieved. It could be because a worker has died.
  UserWarning)


5200/5200 [==============================] - 1990s 383ms/step - loss: 1.0845 - accuracy: 0.5659 - val_loss: 1.3302 - val_accuracy: 0.5385
Epoch 2/100
5200/5200 [==============================] - 371s 71ms/step - loss: 0.7549 - accuracy: 0.7121 - val_loss: 1.9596 - val_accuracy: 0.5913
Epoch 3/100
5200/5200 [==============================] - 372s 72ms/step - loss: 0.6180 - accuracy: 0.7662 - val_loss: 1.6405 - val_accuracy: 0.6247
Epoch 4/100
5200/5200 [==============================] - 370s 71ms/step - loss: 0.5264 - accuracy: 0.8031 - val_loss: 1.2916 - val_accuracy: 0.5553
Epoch 5/100
5200/5200 [==============================] - 366s 70ms/step - loss: 0.4507 - accuracy: 0.8338 - val_loss: 1.3789 - val_accuracy: 0.5707
Epoch 6/100
5200/5200 [==============================] - 366s 70ms/step - loss: 0.3921 - accuracy: 0.8553 - val_loss: 2.2833 - val_accuracy: 0.6350
Epoch 7/100
5200/5200 [==============================] - 367s 71ms/step - loss: 0.3466 - accuracy: 0.8730 - val_loss: 0.98

In [12]:
model.save('./drive/My Drive/Colab Notebooks/Nota_Assignment/mini_XCEPTION_emotion.h5')

In [13]:
print(train_generator.class_indices)

{'anger': 0, 'neutral': 1, 'sad': 2, 'smile': 3, 'surprise': 4}
